In [4]:
from ultralytics import YOLO

# 1. تحميل الموديل المخصص للتصنيف (Suffix -cls)
model = YOLO('yolov8n-cls.pt')

data_path = '/Users/janamac/Documents/project1/parking/clf-data' 

# 3. التدريب
model.train(
    data=data_path,
    epochs=15,
    imgsz=64,
    model='yolov8n-cls.pt'
)

New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.12.5 torch-2.2.2 CPU (Intel Core i5-5350U 1.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/janamac/Documents/project1/parking/clf-data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=6

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


       1/15         0G     0.2444          7         64: 100% ━━━━━━━━━━━━ 305/305 2.2it/s 2:21<0.2s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 39/39 6.5it/s 6.0s0.2s
                   all      0.994          1

      Epoch    GPU_mem       loss  Instances       Size
       2/15         0G    0.04206          7         64: 100% ━━━━━━━━━━━━ 305/305 2.3it/s 2:12<0.2s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 39/39 6.6it/s 5.9s0.1s
                   all      0.996          1

      Epoch    GPU_mem       loss  Instances       Size
       3/15         0G    0.05081          7         64: 100% ━━━━━━━━━━━━ 305/305 2.8it/s 1:47<0.2ss
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 39/39 6.0it/s 6.5s0.1s
                   all      0.976          1

      Epoch    GPU_mem       loss  Instances       Size
       4/15         0G    0.03587          7         64: 100% ━━━━━━━━━━━━ 305/305 2.7it/s 1:52<0.3ss
               classes   to

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x128e82a80>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: PosixPath('/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/runs/classify/train4')
speed: {'preprocess': 0.002120297786450129, 'inference': 2.3563838392111944, 'loss': 4.8893356308133986e-05, 'postprocess': 0.00017075225312225147}
task: 'classify'
top1: 1.0
top5: 1.0

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1. تحميل الموديل والماسك
model = YOLO('/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/runs/classify/train4/weights/best.pt')
mask = cv2.imread('/Users/janamac/Documents/project1/parking/mask_1920_1080.png', cv2.IMREAD_GRAYSCALE)

parking_image = cv2.imread('/Users/janamac/Documents/parking.png')
parking_image = cv2.resize(parking_image, (1920, 1080)) # دي أهم خطوة للضبط

contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # هنقص الركنة مع "هامش" صغير جداً عشان نضمن إننا جوه المربع
    slot_crop = parking_image[y:y+h, x:x+w]
    
    if slot_crop.size > 0:
        # تصنيف الركنة
        results = model(slot_crop, verbose=False)
        probs = results[0].probs
        label = results[0].names[probs.top1]
        conf = probs.top1conf.item() # نسبة التأكد

        # ضبط الحساسية: لو الموديل مش متأكد أوي، اعتبرها "مش معروفة" أو حافظ على اللون
        if conf > 0.7: # بنزود الـ Confidence threshold عشان الدقة
            color = (0, 255, 0) if label == 'empty' else (0, 0, 255)
            cv2.rectangle(parking_image, (x, y), (x+w, y+h), color, 2)

# 4. عرض النتيجة
cv2.imshow('Fixed Detection', parking_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

In [1]:
from ultralytics import YOLO
import cv2

# تحميل موديل YOLOv8 المتدرب على اكتشاف الأشياء (العربيات مثلاً)
model = YOLO('yolov8n.pt') 

# تحميل صورتك الجديدة
img = cv2.imread('/Users/janamac/Documents/project1/archive (1)/boxes/0.png')

# اكتشاف العربيات في الصورة
results = model(img)

# رسم النتائج
for result in results:
    boxes = result.boxes
    for box in boxes:
        # الكلاس رقم 2 في YOLO هو 'car' ورقم 7 هو 'truck'
        if box.cls[0] in [2, 7]: 
            x1, y1, x2, y2 = box.xyxy[0]
            # رسم مربع أحمر حول العربيات المكتشفة
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

cv2.imshow('Detection Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 352x640 1 bus, 11 cell phones, 260.2ms
Speed: 8.7ms preprocess, 260.2ms inference, 7.4ms postprocess per image at shape (1, 3, 352, 640)


: 

In [1]:
import cv2
from ultralytics import YOLO
import os

# السطر ده بيحل مشاكل التعارض في الماك وساعات بيمنع الـ Crash
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# تحميل الموديل والماسك
model = YOLO('/Users/janamac/Documents/project1/parking/best.pt')
mask = cv2.imread('/Users/janamac/Documents/project1/parking/mask_1920_1080.png', cv2.IMREAD_GRAYSCALE)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cap = cv2.VideoCapture('/Users/janamac/Documents/project1/parking/parking_1920_1080_loop.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv2.resize(frame, (1920, 1080))
    empty_slots = 0

    # أهم تعديل: verbose=False عشان ميبعتش آلاف السطور للـ VS Code ويقعه
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 20 and h > 20:
            slot_crop = frame[y:y+h, x:x+w]
            # verbose=False هو السر هنا
            results = model.predict(slot_crop, verbose=False, device='cpu') 
            
            label = results[0].names[results[0].probs.top1]
            color = (0, 255, 0) if label == 'empty' else (0, 0, 255)
            if label == 'empty': empty_slots += 1
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    # عرض النتيجة
    cv2.imshow('Smart Parking Monitor', cv2.resize(frame, (1280, 720)))
    
    # الخروج الآمن
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1) # سطر إضافي للماك

: 